In [33]:
%%capture
%pylab inline --no-import-all
import math 
import utils

utils.set_default_plot_size()

AttributeError: module 'utils' has no attribute 'set_default_plot_size'

In [4]:
from QuantLib import *

In [6]:
today = Date(11, December, 2012)
Settings.instance().evaluationDate = today


In [7]:
# input 3 1-day cash deposit rate
helpers = [ DepositRateHelper(QuoteHandle(SimpleQuote(rate/100)), 
                              Period(1,Days), fixingDays, TARGET(), 
                              Following, False, Actual360()) 
           for rate, fixingDays in [(0.04, 0), (0.04, 1), (0.04, 2)] ]


In [8]:
eonia = Eonia()

In [10]:
#  five OIS forwards on ECB dates
helpers += [ DatedOISRateHelper(start_date, end_date, QuoteHandle(SimpleQuote(rate/100)), eonia) 
             for rate, start_date, end_date in [ 
                 (0.046, Date(16,January,2013), Date(13,February,2013)), 
                 (0.016, Date(13,February,2013), Date(13,March,2013)), 
                 (-0.007, Date(13,March,2013), Date(10,April,2013)), 
                 (-0.013, Date(10,April,2013), Date(8,May,2013)), 
                 (-0.014, Date(8,May,2013), Date(12,June,2013))] ]


In [11]:
# OIS tenors to 30 years
helpers += [ OISRateHelper(2, Period(*tenor), QuoteHandle(SimpleQuote(rate/100)), eonia) 
            for rate, tenor in [(0.002, (15,Months)), (0.008, (18,Months)), 
                                (0.021, (21,Months)), (0.036, (2,Years)), 
                                (0.127, (3,Years)), (0.274, (4,Years)), 
                                (0.456, (5,Years)), (0.647, (6,Years)), 
                                (0.827, (7,Years)), (0.996, (8,Years)), 
                                (1.147, (9,Years)), (1.280, (10,Years)), 
                                (1.404, (11,Years)), (1.516, (12,Years)), 
                                (1.764, (15,Years)), (1.939, (20,Years)), 
                                (2.003, (25,Years)), (2.038, (30,Years))] ]

In [13]:
eonia_curve_c = PiecewiseLogCubicDiscount(0, TARGET(), helpers, Actual365Fixed()) 
eonia_curve_c.enableExtrapolation()


In [16]:
today = eonia_curve_c.referenceDate() 
end = today + Period(2,Years) 
dates = [ Date(serial) for serial in range(today.serialNumber(), end.serialNumber()+1) ] 
rates_c = [ eonia_curve_c.forwardRate(d, TARGET().advance(d,1,Days), Actual360(), Simple).rate() for d in dates ]
utils.plot_curve(plt, dates, [(rates_c,'-')])


RuntimeError: more than one instrument with pillar February 13th, 2013

In [17]:
###### fix turn-of-year jumps
eonia_curve_ff = PiecewiseFlatForward(0, TARGET(), helpers, Actual365Fixed())
eonia_curve_ff.enableExtrapolation()

In [18]:
end = today + Period(6,Months) 
dates = [ Date(serial) for serial in range(today.serialNumber(), end.serialNumber()+1) ]
rates_ff = [ eonia_curve_ff.forwardRate(d, TARGET().advance(d,1,Days), Actual360(), Simple).rate() for d in dates ]

RuntimeError: more than one instrument with pillar February 13th, 2013

In [19]:
utils.plot_curve(plt, dates, [(rates_ff,'-')])

AttributeError: module 'utils' has no attribute 'plot_curve'

In [20]:
nodes = list(eonia_curve_ff.nodes())

RuntimeError: more than one instrument with pillar February 13th, 2013

In [21]:
nodes[:9]

NameError: name 'nodes' is not defined

In [22]:
# need to customize based on the location of the jumps
nodes[6] = (nodes[6][0], (nodes[5][1]+nodes[7][1])/2.0)
nodes[:9]

NameError: name 'nodes' is not defined

In [23]:
temp_dates, temp_rates = zip(*nodes) 
temp_curve = ForwardCurve(temp_dates, temp_rates, eonia_curve_ff.dayCounter())
temp_rates = [ temp_curve.forwardRate(d, TARGET().advance(d,1,Days), Actual360(), Simple).rate() for d in dates ]
utils.plot_curve(plt, dates, [(temp_rates,'-'), (rates_ff,'--')])

NameError: name 'nodes' is not defined

In [24]:
d1 = Date(31,December,2012) - Period(1,Weeks)
d2 = Date(31,December,2012) + Period(1,Weeks)

In [25]:
F = eonia_curve_ff.forwardRate(d1, d2, Actual360(), Simple).rate()
F_1 = temp_curve.forwardRate(d1, d2, Actual360(), Simple).rate() 
print utils.format_rate(F,digits=3), utils.format_rate(F_1,digits=3)

SyntaxError: invalid syntax (<ipython-input-25-01618780c0d3>, line 3)

In [29]:
t12 = eonia_curve_ff.dayCounter().yearFraction(d1,d2)
t_j = eonia_curve_ff.dayCounter().yearFraction(Date(31,December,2012), Date(2,January,2013))
J = (F-F_1)*t12/t_j
print utils.format_rate(J,digits=3)

SyntaxError: invalid syntax (<ipython-input-29-5638ec1da38c>, line 4)

In [30]:
B = 1.0/(1.0+J*t_j)
jumps = [QuoteHandle(SimpleQuote(B))]
jump_dates = [Date(31,December,2012)]
eonia_curve_j = PiecewiseFlatForward(0, TARGET(), helpers, Actual365Fixed(), jumps, jump_dates)


NameError: name 'J' is not defined

In [31]:
rates_j = [ eonia_curve_j.forwardRate(d, TARGET().advance(d,1,Days), 
                                      Actual360(), Simple).rate() for d in dates ]
utils.plot_curve(plt, dates, [(rates_ff,'-'), (rates_j,'o')])

NameError: name 'eonia_curve_j' is not defined

In [32]:
dates = [ today+Period(i,Days) for i in range(0, 365*2+1) ]
rates = [ eonia_curve.forwardRate(d, TARGET().advance(d,1,Days), Actual360(), Simple).rate() for d in dates ]
utils.plot_curve(plt, dates, [(rates,'.')], ymin=-0.001, ymax=0.002)

NameError: name 'eonia_curve' is not defined

In [ ]:
dates = [ today+Period(i,Months) for i in range(0, 12*60+1) ]
rates = [ eonia_curve.forwardRate(d, TARGET().advance(d,1,Days), Actual360(), Simple).rate() for d in dates ]
utils.plot_curve(plt, dates, [(rates,'-')], ymin=0.0, ymax=0.035)